<a href="https://colab.research.google.com/github/sergfer26/Proyectos-II/blob/main/semana6/prueba_vgg_dogs_vs_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Ejercicio] Prueba del modelo con mis propias imágenes

In [ ]:
import torch  
from torch import nn
from PIL import Image
from google.colab import drive 
import torchvision.models as models
from matplotlib import pyplot as plt

## Para utilizar los archivos de Drive

In [ ]:
from google.colab import drive # Colab
drive.mount('/content/drive') # Colab

## Revisar disponibilidad del GPU

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

print(device)

## Visualización de tensores como imágenes 

In [ ]:
def imshow(inp,cmap=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp,cmap)

## Definición del modelo

In [ ]:
vgg = models.vgg16(pretrained=True)
print(vgg)

### *Fine tuning*

In [ ]:
vgg.classifier = nn.Sequential(
    nn.Linear(in_features=25088, out_features=4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 512),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(512, 16),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(16, 1),
    nn.Sigmoid()
)
print(vgg)

### Red neuronal dispoible para gpu

In [ ]:
if torch.cuda.is_available():
  vgg = vgg.cuda()

### Carga de modelo entrenado en clase

Es posible que sea necesario cambiar el `PATH` de donde se encuentra la imagen que usaremos. **Nota:** La red fue entreanada usando GPU, por lo que es necesario cargarla en una red habilitada para usar el GPU, de otra forma marcará error.

In [ ]:
PATH = '/content/drive/MyDrive/Classroom/Proyectos II Grupo 6034/models/vgg-dogs-vs-cats/'
vgg.load_state_dict(torch.load(PATH + 'net'))
vgg.eval() # Restringimos el uso de la red para evaluación

## ¡Carga tu imagen y prueba la red entrenada

In [ ]:
img = Image.open('/content/drive/MyDrive/Classroom/Proyectos II Grupo 6034/data-sets/dogs-vs-cats/dimitri1.JPG')

Transformamos la imagen a tensor.

In [ ]:
img_tensor = simple_transform(img)
print('type: {}, shape: {}'.format(type(img_tensor), img_tensor.size()))

In [ ]:
imshow(img_tensor)

Redimencionamos la imagen de tal forma que la pueda entender la red neuronal.

In [ ]:
batch_tensor = torch.unsqueeze(img_tensor, 0)
print('shape: ', batch_tensor.size())

Probamos el modelo.

In [ ]:
out = vgg(batch_tensor.to(device))
prob = out.item()
print('La imagen con {:.2f} de probilidad es un perro'.format(prob))